In [1]:
Vir = lie_algebras.VirasoroAlgebra(QQ)
M = Vir.verma_module(1/2, 1/2)
d = Vir.basis()
v = M.highest_weight_vector()
m2 = (-4*d[-2] + 3*d[-1]*d[-1])*v
m3 = (-12*d[-2]*d[-1] + 4*d[-1]*d[-1]*d[-1] + 3*d[-3])*v
var('t q')

def L_partition_vector(partition, vector):
    for i in range(len(partition)):
        vector = d[-partition[len(partition) - i - 1]]*vector
    return vector

def my_coefficient(partition, vector):
    return vector.monomial_coefficients().get(tuple(-k for k in partition), 0)

def number_of_ones(partition):
    for i in range(len(partition)):
        if partition[len(partition) - i - 1] != 1:
            return i
    return len(partition)

def my_len(partition):
    return 2*len(partition) - number_of_ones(partition)

def row_to_vector(row, n):
    list_of_partition = ordered_partitions(n)
    return sum(row[i]*L_partition_vector(list_of_partition[i], v) for i in range(len(row)))

def series_t_q(list_of_list_of_partition):
    series = 0
    for i in range(len(list_of_list_of_partition)):
        series_t = 0
        for j in range(len(list_of_list_of_partition[i])):
            series_t = series_t + t^my_len(list_of_list_of_partition[i][j])
        series = series + series_t*q^i
    return series

def partition_ends(partition, tail, is_greater):
    if len(partition) < len(tail) - 1:
        return False
    partition2 = partition[len(partition) - len(tail) + 1: len(partition)]
    if partition2 != tail[1:]:
        return False
    if is_greater:
        if partition == partition2:
            return True
        if partition[len(partition) - len(tail)] > tail[0]:
            return True
    else:
        if partition == partition2:
            return False
        if partition[len(partition) - len(tail)] == tail[0]:
            return True
    return False

def list_of_list_of_partition_ends(list_of_list_of_partition, tail, is_greater):
    return [[partition
             for partition in list_of_partition if partition_ends(partition, tail, is_greater)]
            for list_of_partition in list_of_list_of_partition]

def partition_to_polynomial(partition):
    polynomial = [0]*(sum(partition) - 1)
    for i in range(len(partition) - number_of_ones(partition)):
        polynomial[partition[i] - 2] += 1
    return polynomial

def my_order(partition):
    polynomial = partition_to_polynomial(partition)
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((my_len(partition), len(partition) - number_of_ones(partition), polynomial)) 

def ordered_partitions(n):
    if n <= 0:
        return [[]]
    list_of_partition = list(Partitions(n))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def is_subpartition(partition, partition2):
    if len(partition) < len(partition2):
        return False
    for i in range(len(partition) - len(partition2) + 1):
        if partition[i: i + len(partition2)] == partition2:
            return True
    return False

def partition_contains_any_of_list(partition, list_of_partition):
    for partition2 in list_of_partition:
        if is_subpartition(partition, partition2):
            return True
    return False

def partition_contains_any_of_Q(partition):
    if partition == []:
        return False
    if partition_contains_any_of_list(
        partition,
        [[2], [1, 1, 1], [3, 1, 1], [3, 3], [4, 3, 1], [4, 4, 1], [5, 4, 1, 1], [6, 5, 3, 1]]):
        return True
    if len(partition) > 2:
        for i in range(len(partition) - 2):
            if partition[i] - partition[i + 2] <= 2:
                return True
    if len(partition) > 3:
        for i in range(len(partition) - 3):
            if (partition[i] - partition[i + 3] <= 3
                or (partition[i] - partition[i + 3] <= 4
                    and partition[i + 1] - partition[i + 2] <= 3)):
                return True
    if len(partition) > 4:
        for i in range(len(partition) - 4):
            if (partition[i] - partition[i + 1] == 1 and partition[i + 1] - partition[i + 2] == 2
                and partition[i + 2] - partition[i + 3] == 2
                and partion[i + 3] - partition[i + 4] == 1):
                return True
    return False

def matrix_UVir_m2_m3(n):
    list_of_partition = ordered_partitions(n)
    list_of_partition2 = ordered_partitions(n - 2)
    list_of_partition3 = ordered_partitions(n - 3)
    matrix_m2 = matrix({
        (i, j): my_coefficient(list_of_partition[j], L_partition_vector(list_of_partition2[i], m2))
        for j in range(len(list_of_partition))
        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m3 = matrix({
        (i, j): my_coefficient(list_of_partition[j], L_partition_vector(list_of_partition3[i], m3))
        for j in range(len(list_of_partition))
        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m2 = matrix_m2.stack(matrix_m3)
    matrix_m2 = matrix_m2.rref()
    return matrix_m2

def mW_partition(partition):
    n = sum(partition)
    k = ordered_partitions(n).index(partition)
    A_n = matrix_UVir_m2_m3(n)
    pivots = A_n.pivots()
    try:
        return row_to_vector(A_n.row(pivots.index(k)), n)
    except ValueError:
        return 0

def mW_list_of_partition(list_of_partition):
    return [mW_partition(partition) for partition in list_of_partition]

def pivots_up_to(n):
    list_of_list_of_partition = []
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [list_of_partition[j] for j in matrix_UVir_m2_m3(i).pivots()])
    return list_of_list_of_partition

def basis_up_to(n):
    list_of_list_of_partition = []
    pivots = pivots_up_to(n)
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition if partition not in pivots[i]])
    return list_of_list_of_partition

def basis_up_tov2(n):
    list_of_list_of_partition = []
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition
             if not partition_contains_any_of_Q(partition)])
    return list_of_list_of_partition

In [150]:
gr_G_M = CombinatorialFreeModule(QQ, Partitions(NonNegativeIntegers()));
B = gr_G_M.basis()

def partial0(partition):
    return sum((partition[k] - 1)*B[Partition(sorted(partition[: k] + [partition[k] + 1] + partition[k + 1:], reverse = True))] for k in range(len(partition)))

def partial(element):
    return sum(term[1]*partial0(list(term[0])) for term in element)

def poisson_bracket0(partition):
    return B[Partition(partition + [1])]

def poisson_bracket(element):
    return sum(term[1]*poisson_bracket0(list(term[0])) for term in element)

In [151]:
print(partial(B[Partition([3,3,1])] + B[Partition([4, 3, 2])]))
print(poisson_bracket(-1*B[Partition([3, 2])] + 2*B[Partition([4, 2, 1])]))

4*B[[4, 3, 1]] + B[[4, 3, 3]] + 2*B[[4, 4, 2]] + 3*B[[5, 3, 2]]
-B[[3, 2, 1]] + 2*B[[4, 2, 1, 1]]
